### Before you use the code

You need to install a python package called `pykalman`\
`pip install pykalman`

Don't use the algorithm on large amount of data.
If you want to use it, partition data into small chunks with values on
both sides of the gap and run the partitions individually, and the merge
them back.

In [ ]:
import numpy as np
import pandas as pd
from pykalman import KalmanFilter

# Read the data into a pandas dataframe
df = pd.read_csv('data_gaps.csv')

# Extract the 'HourlyAltimeterSetting' column and convert to numpy array
Qobs = df['HourlyAltimeterSetting'].to_numpy()

# Mask the NaN values
Qobs_masked = np.ma.masked_invalid(Qobs)

print('Before filling the gaps: \n', Qobs_masked)

# Define the Kalman Filter parameters for a 1-dimensional time series
transition_matrices = [1]  # State transition matrix (1D)
observation_matrices = [1]  # Observation matrix (1D)

# Estimate the process and observation covariances
# Use a small value if variance can't be computed due to masked data
if np.ma.var(Qobs_masked):
    observation_covariance = np.ma.var(Qobs_masked)
else:
    observation_covariance = 1e-3  # Small value to prevent errors

transition_covariance = observation_covariance  # Assuming same as observation covariance

# Initialize the Kalman Filter
kf = KalmanFilter(
    transition_matrices=transition_matrices,
    observation_matrices=observation_matrices,
    transition_covariance=transition_covariance,
    observation_covariance=observation_covariance,
    initial_state_mean=Qobs_masked[~Qobs_masked.mask][0],  # First valid observation
    initial_state_covariance=1
)

# Use EM algorithm to estimate parameters
kf = kf.em(Qobs_masked, n_iter=10)

# Perform Kalman smoothing
smoothed_state_means, smoothed_state_covariances = kf.smooth(Qobs_masked)

# Replace the missing values with the smoothed estimates
Qobs_filled = Qobs.copy()
Qobs_filled[np.isnan(Qobs_filled)] = smoothed_state_means[np.isnan(Qobs_filled)].flatten()

print('Predicted values: \n', smoothed_state_means)
print('Data after filling the gaps: \n', Qobs_filled)
